In [1]:
#-----------------------------------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")
import time as t
from datetime import datetime, timedelta, date 
from dateutil.relativedelta import *
from netCDF4 import Dataset   
import glob
import requests 
import pandas as pd
import xarray as xr
import requests 
import functools
import operator
import wget
import os
import gc
import numpy as np
import pygrib
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
import datetime
import holidays
import xarray as xr


import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
import shapefile
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import cartopy, cartopy.crs as ccrs   
import regionmask
from shapely.geometry import Point, Polygon

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker 
import scipy.ndimage as ndimage
from PIL import Image
from glob import glob


#-----METPY---------------------
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:

files= glob('/mnt/c/scripts/temas/stream_plot/dados_grib//psi200_01*.grb2')
li = sorted(files)
li

['/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052500.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052600.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052700.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052800.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052900.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023053000.grb2',
 '/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023053100.grb2']

In [33]:
#Date range de previsões entre as rodadas 

data_atual = datetime.datetime.now().date()

start_time = data_atual + relativedelta(months=1)

end_time = start_time + relativedelta(months=7) 



#Referencias
start =  pd.to_datetime(start_time, format='%Y-%m')

end = pd.to_datetime(end_time, format='%Y-%m')


date_range = list(pd.date_range(start, end, freq='M'))

data_list = []

#--------- Loop------------------------------- 
for q in li:

    membro = q.split("_")[3]

    dataset = xr.open_dataset(q)

    dataset = dataset.assign_coords({'number':membro})
    
    dataset = dataset.swap_dims({"step": "valid_time"}).drop(['step','isobaricInhPa']) #transforma valid_time em dim e retira a coord step
    
    valid_time = dataset.valid_time

    ds = dataset.sel(valid_time = date_range)

    group = ds.groupby('valid_time.month').mean()

    ds = ds.to_dataframe()

    data_list.append(ds)

    print(f" Read: {membro} {q}")
# -------------------------------------------

# Média mensal     

df_concat= pd.concat(data_list, axis=0)

df_mensal = df_concat.groupby(['latitude','longitude','number','valid_time']) #agrupa por lat/lon e valid_time


med_mensal = df_mensal.mean('time') #med de cada rodada 

med_mensal.head(20)


 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052500.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052600.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052700.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052800.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023052900.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023053000.grb2
 Read: 01 /mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_2023053100.grb2
